<a href="https://colab.research.google.com/github/viridis45/Python-data-analysis/blob/master/20190904_6_%ED%85%8D%EC%8A%A4%ED%8A%B8%EC%99%80_%EC%8B%9C%ED%80%80%EC%8A%A4%EB%A5%BC_%EC%9C%84%ED%95%9C_%EB%94%A5%EB%9F%AC%EB%8B%9D(3_3).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 양방향 RNN
 - 자연어 처리에서 흔히 사용됨
 - 순서, 시간에 민감
 - 문장을 이해하는데 단어의 중요성은 단어의 위치에 따라 결정되지 않음 유의
   
  
  
시퀀스를 뒤집으면 성능이 어떻게 달라지는지 알아봅시다

In [4]:
from keras.datasets import imdb
from keras.preprocessing import sequence
from keras import layers
from keras.models import Sequential

max_features = 1000 # 단어수
maxlen = 500 # 사용할 텍스트의 길이

(xtrain, ytrain), (xtest, ytest) = imdb.load_data( num_words = max_features)

xtrain = [x[::-1] for x in xtrain]
xtest = [x[::-1] for x in xtest] # 뒤집음

xtrain = sequence.pad_sequences(xtrain, maxlen=maxlen)
ytest = sequence.pad_sequences(xtest, maxlen=maxlen)

model = Sequential()
model.add(layers.Embedding(max_features, 128))
model.add(layers.LSTM(32))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(xtrain, ytrain, epochs=10, batch_size=128, validation_split=0.2)


W0905 01:42:06.953984 139667791185792 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:1033: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================] - 112s 6ms/step - loss: 0.5660 - acc: 0.7128 - val_loss: 0.4633 - val_acc: 0.7884
Epoch 2/10
20000/20000 [==============================] - 110s 5ms/step - loss: 0.4326 - acc: 0.8161 - val_loss: 0.5535 - val_acc: 0.7374
Epoch 3/10
20000/20000 [==============================] - 110s 6ms/step - loss: 0.4079 - acc: 0.8282 - val_loss: 0.5421 - val_acc: 0.7718
Epoch 4/10
20000/20000 [==============================] - 111s 6ms/step - loss: 0.3785 - acc: 0.8437 - val_loss: 0.3816 - val_acc: 0.8360
Epoch 5/10
20000/20000 [==============================] - 110s 5ms/step - loss: 0.3613 - acc: 0.8536 - val_loss: 0.4087 - val_acc: 0.8208
Epoch 6/10
20000/20000 [==============================] - 112s 6ms/step - loss: 0.3544 - acc: 0.8565 - val_loss: 0.3734 - val_acc: 0.8352
Epoch 7/10
20000/20000 [==============================] - 111s 6ms/step - loss: 0.3350 - acc: 0.8650 - val_loss: 0.3852 - v

정방향과 동일한 성능 == 언어를 이해하는데에 순서가 중요하지만 결정적이지는 않다

- 고로 양뱡향 rnn.
- 하나씩 뒤집는듯


In [7]:
model = Sequential()
model.add(layers.Embedding(max_features, 32))
model.add(layers.Bidirectional(layers.LSTM(32)))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(xtrain, ytrain, epochs=10, batch_size=128, validation_split=0.2)


#위를 이전에 보았던 온도데이터에 접목시켜본다

Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================] - 154s 8ms/step - loss: 0.6054 - acc: 0.6684 - val_loss: 0.4843 - val_acc: 0.8022
Epoch 2/10
20000/20000 [==============================] - 149s 7ms/step - loss: 0.4476 - acc: 0.8082 - val_loss: 0.3791 - val_acc: 0.8478
Epoch 3/10
20000/20000 [==============================] - 147s 7ms/step - loss: 0.4004 - acc: 0.8347 - val_loss: 0.4658 - val_acc: 0.7830
Epoch 4/10
20000/20000 [==============================] - 147s 7ms/step - loss: 0.3798 - acc: 0.8466 - val_loss: 0.3882 - val_acc: 0.8370
Epoch 5/10
20000/20000 [==============================] - 148s 7ms/step - loss: 0.3753 - acc: 0.8478 - val_loss: 0.3825 - val_acc: 0.8448
Epoch 6/10
20000/20000 [==============================] - 148s 7ms/step - loss: 0.3579 - acc: 0.8570 - val_loss: 0.4233 - val_acc: 0.8286
Epoch 7/10
20000/20000 [==============================] - 148s 7ms/step - loss: 0.3500 - acc: 0.8572 - val_loss: 0.4452 - v

### 막간을 이용한 Embedding 리뷰 한번 더 [1](https://machinelearningmastery.com/use-word-embedding-layers-deep-learning-keras/) [2](https://www.kaggle.com/rajmehra03/a-detailed-explanation-of-keras-embedding-layer)
- dense vector representation계
- traditional BOW : uses large space vec where it's mostly filled with zeros -- sparse
- Embedding == projection of word into continuous vec space
- based on words surrounding the target word
- int for each words, initialized with a random weight



### 온도 예측모델의 성능을 향상시키려면
1. 스태킹 유닛 수 조절
2. RMSprop 학습률 조절
3. LSTM 사용
4. 유닛 수가 많은 Dense층을 스태킹
5. 최종모델을 테스트세트에서 확인. 검증세트에 과대적합하는것을 방지하기 위해.


### 리뷰
1. 상식 수준의 기준점에서 시작하자
2. 간단한 모델에서 먼저 시도
3. 시계열이라면 순환층
4. dropout and recurrent_dropout
5. 스태킹 RNN로 overkill하지는 말자
6. 양방향 RNN은 최근 정보가 더 중요한 데이터에는 부적합


# 컨브넷을 사용한 시퀀스 처리
- 인풋의 특성을 뽑아내는 컨브넷(합성곱연산)을 시퀀스 처리에 사용해보자
- 1D 컨브넷 :
  - 특정 시퀀스에서 RNN 대체. 
  - 적은 비용
  - 오디오 생성, 기계 번역에서 대세
  
### 1D합성곱 이해하기
- 2D합성곱 어떻게 했는지 기억하니 : 이미지텐서에서 2d패치 추출 후 모든 패치에 동일한 변환 적용
- 이걸 1d에 적용한다.
- 똑같음. 특정 위치의 패턴이 재발하는것을 인식. 윈도우.
- subsampling 하는데에 똑같이 평균풀링, 맥스풀링 적용


### 1D 합성곱 쌓기
- 2D와 같음.
1. Conv1D / MaxPooling1D 쌓기
2. GlobalAveragePooling1D / GlobalMaxPooling1D / Flatten으로 마무리
- 위는 (samples, timesteps, features)를 입력받고 3d
- (samples, features) 를 돌려준다 2d
3. GlobalAverage'2D'Pooling 은 4d - 2d : (sample, h, w, channels) -- (samples, channels)



In [13]:
# did this above but doing it again for muscle memory

from keras.datasets import imdb
from keras.preprocessing import sequence
from keras.optimizers import RMSprop

max_features = 1000
max_len = 100

print('loading data')
(xtrain, ytrain),(xtest, ytest) = imdb.load_data(num_words = max_features)
print(len(xtrain), 'training sequence')
print(len(xtest), 'test sequence')

print('\nsequence padding (samples x time)')
xtrain = sequence.pad_sequences(xtrain, maxlen=max_len)
xtest = sequence.pad_sequences(xtest, maxlen=max_len)

print('x_train size: ', xtrain.shape)
print('x_test size: ', xtest.shape)


# now the convnet
model = Sequential()
model.add(layers.Embedding(max_features, 128, input_length = max_len))
model.add(layers.Conv1D(32, 7, activation = 'relu'))
model.add(layers.MaxPooling1D(5))
model.add(layers.Conv1D(32,7,activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(1))

model.summary()


model.compile(optimizer = RMSprop(lr=1e-4), loss = 'binary_crossentropy', metrics=['acc'])

history = model.fit(xtrain, ytrain, epochs = 10, batch_size = 128, validation_split = 0.2)

loading data
25000 training sequence
25000 test sequence

sequence padding (samples x time)
x_train size:  (25000, 100)
x_test size:  (25000, 100)
Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_12 (Embedding)     (None, 100, 128)          128000    
_________________________________________________________________
conv1d_9 (Conv1D)            (None, 94, 32)            28704     
_________________________________________________________________
max_pooling1d_6 (MaxPooling1 (None, 18, 32)            0         
_________________________________________________________________
conv1d_10 (Conv1D)           (None, 12, 32)            7200      
_________________________________________________________________
global_max_pooling1d_3 (Glob (None, 32)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 1)

### CNN과 RNN연결하여 긴 시퀀스를 처리하기
- 컨브넷은 인풋을 독립적으로 처리 -- 타임스텝의 순서에 민감하지 않음
- conv1d를 RNN의 전처리용으로 사용하여 경량성을 응용
    - 긴 시퀀스의 데이터 경량화

